In [71]:
#As seen from the results below, this table is unwieldy and difficult to use for our requirments. 
#Therefore, this script/notebook contains scripts to convert the source data into a form we like and as described in our reports.
#The data we get from this notebook is used to populate our database

In [72]:
import pandas as pd
from numpy import nan

In [73]:
df = pd.read_csv('MCI_2014_to_2019.csv')
btDF = pd.read_csv('BikeTheft_raw.csv')
pdDF = pd.read_csv('PoliceDivision_raw.csv')

In [74]:
print(df.columns.tolist())
print(btDF.columns.tolist())
print(pdDF.columns.tolist())

['X', 'Y', 'Index_', 'event_unique_id', 'occurrencedate', 'reporteddate', 'premisetype', 'ucr_code', 'ucr_ext', 'offence', 'reportedyear', 'reportedmonth', 'reportedday', 'reporteddayofyear', 'reporteddayofweek', 'reportedhour', 'occurrenceyear', 'occurrencemonth', 'occurrenceday', 'occurrencedayofyear', 'occurrencedayofweek', 'occurrencehour', 'MCI', 'Division', 'Hood_ID', 'Neighbourhood', 'Long', 'Lat', 'ObjectId']
['X', 'Y', 'FID', 'Index_', 'event_unique_id', 'Primary_Offence', 'Occurrence_Date', 'Occurrence_Year', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Time', 'Division', 'City', 'Location_Type', 'Premise_Type', 'Bike_Make', 'Bike_Model', 'Bike_Type', 'Bike_Speed', 'Bike_Colour', 'Cost_of_Bike', 'Status', 'Hood_ID', 'Neighbourhood', 'Lat', 'Long']
['OBJECTID', 'AGENCY', 'DIV', 'UNIT_NAME', 'ADDRESS', 'CITY', 'AREA_SQKM', 'Shape_Leng', 'Shape__Area', 'Shape__Length']


In [75]:
df2_1 = df[[ "occurrencehour", "occurrenceday","occurrencemonth", "occurrenceyear", "occurrencedayofweek"]]
btDFTemp = btDF[["Occurrence_Time", "Occurrence_Day", "Occurrence_Month", "Occurrence_Year"]]
btDFTemp['Occurrence_Time'] = [i.split(':')[0] for i in btDFTemp['Occurrence_Time']]
btDFTemp.insert(loc=4, column="occurrencedayofweek", value=None)
btDFTemp.rename(columns={"Occurrence_Time":"occurrencehour", "Occurrence_Day":"occurrenceday", "Occurrence_Month":"occurrencemonth", "Occurrence_Year":"occurrenceyear"}, inplace=True)
df2_1 = df2_1.append(btDFTemp)
df2_2 = df[["reportedhour", "reportedday","reportedmonth", "reportedyear", "reporteddayofweek"]]
df2_1.rename(columns={"occurrencehour":"hour", "occurrenceday":"day", "occurrencemonth":"month", "occurrenceyear":"year", "occurrencedayofweek":"day_of_week"}, inplace=True)
df2_2.rename(columns={"reportedhour":"hour", "reportedday":"day", "reportedmonth":"month", "reportedyear":"year", "reporteddayofweek":"day_of_week"}, inplace=True)
IncidentTime = df2_1.append(df2_2)
IncidentTime = IncidentTime.drop_duplicates()
IncidentTime = IncidentTime.rename_axis('time_id').reset_index()
IncidentTime['time_id'] += 600000
IncidentTime.to_csv("IncidentTime_prod.csv", index=False)
print(IncidentTime)

/home/dragonemperor/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dragonemperor/.local/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


       time_id hour   day      month    year day_of_week
0       600000    1  24.0      March  2014.0  Monday    
1       600001   16  27.0  September  2014.0  Saturday  
2       600002    6  24.0      March  2014.0  Monday    
3       600003   15  24.0      March  2014.0  Monday    
4       600004    2   3.0        May  2014.0  Saturday  
...        ...  ...   ...        ...     ...         ...
66495   805289    5  27.0      April  2018.0  Friday    
66496   805290    7  27.0      April  2018.0  Friday    
66497   805291    6  27.0      April  2018.0  Friday    
66498   805949    6  21.0      March  2019.0  Thursday  
66499   806309    6   8.0     August  2019.0  Thursday  

[66500 rows x 6 columns]


In [76]:
PoliceDivision = pdDF[["DIV", "ADDRESS"]]
PoliceDivision.rename(columns={"DIV":"division", "ADDRESS":"address"}, inplace=True)
PoliceDivision.to_csv("PoliceDivision_prod.csv", index=False)
print(PoliceDivision)

   division             address
0       D11   2054 Davenport Rd
1       D12    200 Trethewey Dr
2       D13  1435 Eglinton Av W
3       D14   350 Dovercourt Rd
4       D22     3699 Bloor St W
5       D23     5230 Finch Av W
6       D31      40 Norfinch Dr
7       D32     30 Ellerslie Av
8       D33        50 Upjohn Rd
9       D41  2222 Eglinton Av E
10      D42     242 Milner Av E
11      D43  4331 Lawrence Av E
12      D51    51 Parliament St
13      D52     255 Dundas St W
14      D53    75 Eglinton Av W
15      D54     41 Cranfield Rd
16      D55      101 Coxwell Av


/home/dragonemperor/.local/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [77]:
Neighbourhood = df[["Hood_ID", "Neighbourhood"]]
Neighbourhood = Neighbourhood.append(btDF[["Hood_ID", "Neighbourhood"]])
Neighbourhood = Neighbourhood.drop_duplicates()
Neighbourhood.rename(columns={"Hood_ID":"hood_id", "Neighbourhood":"name"}, inplace=True)
Neighbourhood['name'] = [i.split('(')[0] for i in Neighbourhood['name']]
Neighbourhood.to_csv("Neighbourhood_prod.csv", index=False)
print(Neighbourhood)

      hood_id                     name
0         132                 Malvern 
1          76     Bay Street Corridor 
2           1  West Humber-Clairville 
3          47      Don Valley Village 
4          90           Junction Area 
...       ...                      ...
1051        4         Rexdale-Kipling 
1218       23    Pelmo Park-Humberlea 
1302      106      Humewood-Cedarvale 
1337       99     Mount Pleasant East 
2538       49   Bayview Woods-Steeles 

[140 rows x 2 columns]


In [78]:
BikeTheft = btDF[["Bike_Colour", "Bike_Make", "Bike_Model", "Bike_Speed", "Bike_Type", "Status", "Cost_of_Bike"]]
BikeTheft.rename(columns={"Bike_Colour":"colour", "Bike_Make":"make", "Bike_Model":"model", "Bike_Speed":"speed", "Bike_Type":"bike_type", "Status":"status", "Cost_of_Bike":"cost"}, inplace=True)
BikeTheft = BikeTheft.rename_axis('bike_theft_id').reset_index()
BikeTheft['bike_theft_id'] += 600000
BikeTheft.to_csv("BikeTheft_prod.csv", index=False)
print(BikeTheft)

/home/dragonemperor/.local/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


       bike_theft_id  colour             make            model  speed  \
0             600000  ONG     SUPERCYCLE                   NaN     10   
1             600001  BLK     ROCKY MOUNTAIN   TRAILHEAD            1   
2             600002  BLU     MONGOOSE                     NaN     18   
3             600003  WHI     SCOTT                        NaN     24   
4             600004  BLU     RALEIGH                      NaN     18   
...              ...     ...              ...              ...    ...   
21579         621579  BLK     CC               CCM APEX WOMEN'     24   
21580         621580  LBL     GI               2016 ALIGHT 3       12   
21581         621581  ONG     NO               2017 THRESHOLD      22   
21582         621582  BLK     NA               ROYAL               18   
21583         621583  BLK     TR               MARLIN 6             1   

      bike_type     status     cost  
0            MT     STOLEN      NaN  
1            MT  RECOVERED      NaN  
2        

In [79]:
df4 = df[[ "offence", "MCI"]]
df4 = df4.drop_duplicates()
df4 = df4.rename_axis('crimeID').reset_index()
df4['crimeID'] += 100000
df4.to_csv("RegularCrime_prod.csv", index=False)
print(df4)

    crimeID                         offence              MCI
0    100000                         Assault          Assault
1    100001                             B&E  Break and Enter
2    100004              Robbery - Business          Robbery
3    100005             Assault With Weapon          Assault
4    100008                      Theft Over       Theft Over
5    100009               Robbery - Mugging          Robbery
6    100011        Theft Over - Shoplifting       Theft Over
7    100026                    B&E W'Intent  Break and Enter
8    100035              Aggravated Assault          Assault
9    100040             Assault Bodily Harm          Assault
10   100042  Assault - Resist/ Prevent Seiz          Assault
11   100056                 Robbery - Other          Robbery
12   100061             Robbery With Weapon          Robbery
13   100065           Assault Peace Officer          Assault
14   100072         Robbery - Home Invasion          Robbery
15   100091    Unlawfull

In [80]:
df5 = df.merge(df4, on=['offence','MCI'] )
df6 = pd.merge(df5, Neighbourhood, left_on='Hood_ID', right_on='hood_id')
df6.rename(columns={"occurrencehour":"hour", "occurrenceday":"day", "occurrencemonth":"month", "occurrenceyear":"year", "occurrencedayofweek":"day_of_week"}, inplace=True)
df7 = df6.merge(IncidentTime, on=["hour", "day", "month", "year", "day_of_week"])
df13 = IncidentTime.copy()
df13.rename(columns={'time_id': 'reportedTimeID', "hour":"reportedhour", "day":"reportedday", "month":"reportedmonth", "year":"reportedyear", "day_of_week":"reporteddayofweek"}, inplace=True)
df14 = df13.merge(df7, on=["reportedhour", "reportedday","reportedmonth", "reportedyear", "reporteddayofweek"])
df14 = df14[['event_unique_id','crimeID','Hood_ID','time_id','reportedTimeID', "Long", "Lat","premisetype"]]
print(df14)
df14.to_csv("CrimeEvent_prod.csv", index=False)

       event_unique_id  crimeID  Hood_ID  time_id  reportedTimeID       Long  \
0       GO-20141756091   100000       31   600000          600000 -79.450104   
1       GO-20141755910   100000        2   600986          600000 -79.581047   
2       GO-20141755910   100000        2   600986          600000 -79.581047   
3       GO-20142996126   100241        2   600001          600001 -79.589233   
4       GO-20142995995   100000      113   600001          600001 -79.504250   
...                ...      ...      ...      ...             ...        ...   
206430   GO-2019510013   157000       26   748581          805949 -79.522148   
206431  GO-20191496247   157000      127   734808          806309 -79.246758   
206432  GO-20191496261   157000       36   741119          806309 -79.421677   
206433  GO-20191496261   157000       36   741119          806309 -79.421677   
206434  GO-20191496218   157000       36   741912          806309 -79.439766   

              Lat premisetype  
0      